In [1]:
import numpy as np
from scipy.linalg import toeplitz
from scipy.stats import logistic
from BORR import Bayesian_Ordinal_CLM_PO

In [2]:
N_train=360
N_test=100
Block_size=50
P=500
Q=100
Corr=0.9**toeplitz(np.arange(0,Block_size),np.arange(0,Block_size))
Corr_random=0.000001**toeplitz(np.arange(0,Q),np.arange(0,Q))
Corr_b=0.2**toeplitz(np.arange(0,Q),np.arange(0,Q))

np.random.seed(5041294)

X_train=np.ones((N_train,P))
X_test=np.ones((N_test,P))
Y_train=np.ones((N_train,1))
Y_test=np.ones((N_test,1))
T_train=np.random.multivariate_normal(np.zeros(Q), Corr_random/Q, N_train)
T_test=np.random.multivariate_normal(np.zeros(Q), Corr_random/Q, N_test)
beta=np.zeros((P,1))
error_train=logistic.rvs(loc=0, scale=1, size=N_train)
error_test=logistic.rvs(loc=0, scale=1, size=N_test)
#error_train=np.random.randn(N_train,1)
#error_test=np.random.randn(N_test,1)

for i in range(0,6):
    
    X_train[:,50*i:(50*(i+1))]=np.random.multivariate_normal(np.zeros(Block_size), Corr, N_train)
    X_test[:,50*i:(50*(i+1))]=np.random.multivariate_normal(np.zeros(Block_size), Corr, N_test)

beta[0]=-np.log(3)
beta[30]=-np.log(3)
beta[60]=-np.log(3)
beta[90]=-np.log(3)
beta[120]=-np.log(3)
beta[150]=np.log(3)
beta[180]=np.log(3)
beta[210]=np.log(3)
beta[240]=np.log(3)
beta[270]=np.log(3)

random_effect_train=np.zeros((N_train,1))
random_effect_test=np.zeros((N_test,1))

for i in range(0,N_train):
    
    random_effect_train[i]=T_train[i,]@(np.random.multivariate_normal(np.zeros(Q), Corr_b, 1).T)
    
for i in range(0,N_test):
    
    random_effect_test[i]=T_test[i,]@(np.random.multivariate_normal(np.zeros(Q), Corr_b, 1).T)
    
#random_effect_train=np.zeros((N_train,1))
#random_effect_test=np.zeros((N_test,1))    
     
Z_train=X_train@beta+random_effect_train+error_train.reshape(N_train,1)
Z_test=X_test@beta+random_effect_test+error_test.reshape(N_test,1)

Y_train[Z_train>np.quantile(Z_train, 0.167)]=2
Y_train[Z_train>np.quantile(Z_train, 0.333)]=3
Y_train[Z_train>np.quantile(Z_train, 0.5)]=4
Y_train[Z_train>np.quantile(Z_train, 0.667)]=5
Y_train[Z_train>np.quantile(Z_train, 0.833)]=6

Y_test[Z_test>np.quantile(Z_test, 0.167)]=2
Y_test[Z_test>np.quantile(Z_test, 0.333)]=3
Y_test[Z_test>np.quantile(Z_test, 0.5)]=4
Y_test[Z_test>np.quantile(Z_test, 0.667)]=5
Y_test[Z_test>np.quantile(Z_test, 0.833)]=6

In [3]:
beta_sample,b_sample,cutpoints_sample=Bayesian_Ordinal_CLM_PO(Y_train,X_train,T_train)

In [4]:
beta_mean=np.mean(beta_sample,axis=1)

In [16]:
beta_mean[np.abs(beta_mean)/np.std(beta_sample,axis=1)>1.2]

array([-0.55810848, -0.51632061, -1.09348665, -0.44563244, -1.14614348,
       -0.40605648,  0.59454806,  0.61926416,  0.64811042,  0.51503447,
        0.91136733])

In [15]:
np.where(np.abs(beta_mean)/np.std(beta_sample,axis=1)>1.2)

(array([  0,   1,  30,  60,  90, 120, 150, 180, 210, 240, 270]),)